TODO:

- Remover o código assim que ficar finalizado e passado para a pasta scripts
- Escrever testes e documentação de funções
- 

# Gibbs Sampling, o que é?

É um algoritmo do tipo MCMC (Markov Chain monte Carlo) tipicamente utilizado para extrair amostras de distribuições de probabilidade n-dimensionais, facilitando a amostragem.

É uma técnica que serve para obter uma sequência de observações (neste caso uma sub-sequências de DNA).

# Como serve para encontrar *motifs*?

Serve para encontrar o padrão/*motif* mais comum a um conjunto de sequências partindo de um conjunto aleatório de padrões.

# Como é aplicado?

0. Considerando um conjunto de T sequências de tamanho N, é gerada a sua PWM

1. Computamos o subtotal de todas as sequências em função do PWM multiplicando a ocorrência de cada nucleótido pela sua probabilidade na PWM.

2. É procurada a sub-sequência de tamanho L na sequência S<sub>i</sub> a partir da posição P (P $\le$ N-L), em que P é selecionada aleatoriamente em função da distribuição de probabilidades e score total da sequência - P(motif|score da sequência). É computado o *score* do *motif* em função da matriz de probabilidade (PWM)

3. Repetimos este passo para todas as restantes sequências (S<sub>i+1</sub>,...,S<sub>t</sub>), atualizando o melhor motif caso o score dos motifs identificados for superior ao melhor atual.

4. São feitas sucessivas iterações até convergir para um *motif* mais comum ao conjunto de todas as sequências S (S<sub>i</sub>,...,S<sub>i+t</sub>) e encontrar as melhores posições.


In [ ]:
# Conjunto de sequências
seqs = 'aaaaaa aaaaaa aaaaaa gggggg'.split()

'''
Criamos a PWM de forma rápida

Neste caso usamos uma lista por compreensão com recurso ao método zip(*seqqs) que
irá emparelhar as sequências posição a posição e faz a contagem da base / pelo total

Incluímos já a consideração de pseudocontagens
'''

'\nCriamos a PWM de forma rápida\n\nNeste caso usamos uma lista por compreensão com recurso ao método zip(*seqqs) que\nirá emparelhar as sequências posição a posição e faz a contagem da base / pelo total\n\nIncluímos já a consideração de pseudocontagens\n'

In [ ]:
from random import randint as ri
from random import choice

# Gerar sequências aleatoriamente para teste
seqs = new_seqs = [''.join(choice('ACGT') for _ in range(20)) for i in range(10)]

In [11]:
def gerar_seqs(n,t):
  from random import choice
  return [''.join(choice('ACGT') for p in range(n)) for i in range(t)]


In [12]:
'''
Selecionamos posições aleatórias para cada sequência, tendo em atenção o
tamanho do nosso motif
'''

# Declaramos o tamanho do nosso motif
L = 8

# Declaramos t, que corresponde ao número de sequências que temos
t = len(new_seqs) -1

# Declaramos n, que corresponde ao tamanho das nossas sequências
# assumindo que são todas de tamanho igual
n = len(new_seqs[0])

# Declaramos o valor máximo de offset que podemos atingir
offset_max = n - L

In [13]:
# De seguida escolhemos uma sequência aleatoriamente da nossa lista de sequências
# para analisar a probabilidade de cada um dos motifs ocorrer nessa sequência
# Atentar no len(lista) - 1 para garantir que estamos sempre no índice

def escolhe_seq(seqs):
  from random import choice
  return choice(list(enumerate(seqs)))


In [49]:
# Criamos um vetor com as posições selecionadas aleatoriamente
# E criamos os snips para cada um deles
from random import randint as ri
from random import choice

# Gerar sequências aleatoriamente para teste
seqs = gerar_seqs(50,10)

indice, escolhida = escolhe_seq(seqs)

# Função para gerar snips aleatórios de sequências exceto a escolhida
def gerar_snips(seqs , L , offset_max, indice):

  from random import randint as ri

  offsets = [ri(0,offset_max) for pos, seq in enumerate(seqs) if pos != indice]
  snips = [seq[pos: pos + L] for seq, pos in zip(seqs,offsets) if seq != seq[indice]]

  return offsets, snips

offsets, snips = gerar_snips(seqs, L , offset_max, indice)
#len(snips)

In [57]:
sum([max({base: snip.count(base) for base in 'ACGT'}.values()) for snip in snips])

34

[{'A': 0.15384615384615385,
  'C': 0.23076923076923078,
  'G': 0.3076923076923077,
  'T': 0.3076923076923077},
 {'A': 0.15384615384615385,
  'C': 0.15384615384615385,
  'G': 0.23076923076923078,
  'T': 0.46153846153846156},
 {'A': 0.15384615384615385,
  'C': 0.46153846153846156,
  'G': 0.07692307692307693,
  'T': 0.3076923076923077},
 {'A': 0.38461538461538464,
  'C': 0.23076923076923078,
  'G': 0.07692307692307693,
  'T': 0.3076923076923077},
 {'A': 0.23076923076923078,
  'C': 0.23076923076923078,
  'G': 0.3076923076923077,
  'T': 0.23076923076923078},
 {'A': 0.23076923076923078,
  'C': 0.23076923076923078,
  'G': 0.46153846153846156,
  'T': 0.07692307692307693},
 {'A': 0.23076923076923078,
  'C': 0.07692307692307693,
  'G': 0.3076923076923077,
  'T': 0.38461538461538464},
 {'A': 0.38461538461538464,
  'C': 0.23076923076923078,
  'G': 0.23076923076923078,
  'T': 0.15384615384615385}]

In [35]:
# Criamos a PWM para os motizfs, excluindo o proveniente da nossa
# string escolhida e encontramos o consenso

def pwm(seqs , pseudo = 1):
  # Criamos uma lista por compreensão que contém 1 dicionário de P(base) por sequência
  return [
      {base: (seq.count(base) + pseudo) / (len(seq) + 4 * pseudo) for base in 'ACGT'} for seq in zip(*seqs)
      ]

P = pwm(snips)
#consenso = ''.join(max(pos, key=pos.get) for pos in P)
P 

[{'A': 0.15384615384615385,
  'C': 0.23076923076923078,
  'G': 0.3076923076923077,
  'T': 0.3076923076923077},
 {'A': 0.15384615384615385,
  'C': 0.15384615384615385,
  'G': 0.23076923076923078,
  'T': 0.46153846153846156},
 {'A': 0.15384615384615385,
  'C': 0.46153846153846156,
  'G': 0.07692307692307693,
  'T': 0.3076923076923077},
 {'A': 0.38461538461538464,
  'C': 0.23076923076923078,
  'G': 0.07692307692307693,
  'T': 0.3076923076923077},
 {'A': 0.23076923076923078,
  'C': 0.23076923076923078,
  'G': 0.3076923076923077,
  'T': 0.23076923076923078},
 {'A': 0.23076923076923078,
  'C': 0.23076923076923078,
  'G': 0.46153846153846156,
  'T': 0.07692307692307693},
 {'A': 0.23076923076923078,
  'C': 0.07692307692307693,
  'G': 0.3076923076923077,
  'T': 0.38461538461538464},
 {'A': 0.38461538461538464,
  'C': 0.23076923076923078,
  'G': 0.23076923076923078,
  'T': 0.15384615384615385}]

In [ ]:
def pwm(seqs , pseudo = 1):
  # Criamos uma lista por compreensão que contém 1 dicionário de P(base) por sequência
  
  
  
  return [
      {base: (seq.count(base) + pseudo) / (len(seq) + 4 * pseudo) for base in 'ACGT'} for seq in zip(*seqs)
      ]

In [36]:
[[0 for base in 'ACGT'] for _ in range(len(seqs))]

[[seq.count(base) + pseudo /(len(seq) + 4 * pseudo) for base in 'ACGT'] for seq in seqs]


NameError: name 'pseudo' is not defined

In [39]:
len(snips[0]), len(P)

(8, 8)

In [48]:
# Calculamos P(snip|P) para cada snip
# Isto é, a probabilidade de cada snip ser gerado por P

def prob_snip(snip, P):
  '''
  Função que calcula a P(snip|P), para determinar a probabilidade
  deste ser gerado pelo perfil P

  Parametros:

  snip : str
    strings correspondente a bases de DNA

  P : list[dict]
    perfil probabilistico para um conjunto de sub-sequências de DNA

  Devolve:

  float
    P(snip|P)
  '''
  score = 1
  for pos , base in enumerate(snip):
    score *= P[pos][base]
  return score

from random import choices
teste = [ list(c.values()) for c in P ]



8

In [82]:
for _ in range(len(escolhida) - L + 1):
    motif = escolhida[_ : _ + L]
    prob = 1
    for __ in range(L):
        prob *= P[__][motif[__]] / 10
        #print(prob)


res = []
for offset in range(len(escolhida) - L + 1):

    motif = escolhida[offset : offset + L]

    prob = 1

    for pos in range(L):

        prob *= P[pos][motif[pos]] / 10 # -> será o nosso T

    res.append(prob)

res

[5.295865276109911e-14,
 6.619831595137389e-14,
 4.413221063424926e-15,
 2.647932638054955e-14,
 8.826442126849853e-14,
 2.6479326380549555e-14,
 7.14941812274838e-13,
 8.473384441775857e-14,
 5.95784843562365e-13,
 1.0591730552219822e-13,
 2.1183461104439644e-13,
 1.129784592236781e-13,
 5.648922961183905e-14,
 1.0591730552219824e-14,
 2.118346110443965e-14,
 3.177519165665947e-14,
 2.1183461104439649e-13,
 1.2710076662663787e-13,
 1.1768589502466468e-14,
 4.964873696353042e-15,
 4.413221063424926e-14,
 1.9065114993995681e-13,
 2.259569184473562e-13,
 1.271007666266379e-13,
 1.0591730552219822e-13,
 1.7652884253699706e-14,
 5.2958652761099106e-15,
 2.978924217811825e-14,
 9.532557496997841e-14,
 1.0591730552219822e-13,
 1.0591730552219824e-13,
 4.4132210634249266e-15,
 3.57470906137419e-14,
 9.532557496997841e-14,
 8.473384441775857e-14,
 1.7873545306870952e-12,
 7.061153701479881e-15,
 4.7662787484989203e-14,
 7.061153701479881e-15,
 3.9718989570824336e-13,
 4.7662787484989203e-14,
 

In [17]:

# Calculamos as probabilidades de modo a escolher a posição p com
# melhor probabilidade de geração do motif

def best_pos(snips, offsets, P):

  from random import choices

  # Calculamos P(snip|P) para cada snip
  probs = [prob_snip(snip, P) for snip in snips]

  # Calculamos a probabilidade combinada para cada snip
  prob_comb = [round(prob/sum(probs),5) for prob in probs]

  # Garantimos que a soma totaliza 1
  assert round(sum(prob_comb),0) == 1

  # Devolve a melhor probabilidade e o offset associado a este
  return max(list(zip(prob_comb,offsets)))

#melhor_prob, melhor_offset = best_pos(snips,offsets,P)
#snips[melhor_offset]

In [40]:
from random import choices

choices(snips, P)

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [18]:
#iter = 0

def gibbs_sampler(seqs,L,iter, threshold):
  '''
  Add docstrings
  '''
  i_zero = iter
  # Garantir que todas as sequências têm o mesmo tamanho
  assert all(len(seq) == len(seqs[0]) for seq in seqs)

  # Definir n como o tamanho das nossas sequências
  n = len(seqs[0])

  # Definir o valor de posição máximo possível (n - L)
  offset_max = n - L

  # Inicialização das variáveis-resultado
  best_so_far = -1
  best_offset_so_far = -1
  best_motif_so_far = ''
  best_motifs = []

  # Iniciamos o loop de iterações em função do número indicado ou do threshold
  while iter > 0 and best_so_far <= threshold:
    #print(best_motif_so_far, best_so_far, best_offset_so_far)

    # Escolhemos uma sequência aleatória
    indice, escolhida = escolhe_seq(seqs)

    # Determinamos snips e offsets para as restantes
    offsets, snips = gerar_snips(seqs, L, offset_max, indice)
    #print(snips)

    # Geramos a matriz PWM
    P = pwm(snips)

    # Vemos a posição p com maior probabilidade e respetivo offset
    best_p, best_o = best_pos(snips, offsets, P)
    #print(best_p, best_o)

    # Comparamos com o "best so far" e atualizamos
    if best_p > best_so_far:
      best_so_far = best_p
      best_offset_so_far = best_o
      best_motif_so_far = escolhida[best_o:best_o+L]

      # Adicionamos o motif à lista de motifs candidatos
      best_motifs.append((
        f'S{str(indice)}',
        f'p = {str(best_offset_so_far)}',
        best_motif_so_far,
        best_so_far))



    iter -=1

  print(f'Threshold {threshold} atingido ao fim de {i_zero-iter} iterações.')
  return best_motifs



NameError: name 'P' is not defined

In [19]:
import time
start = time.time()
seqs = gerar_seqs(50,20)
res = gibbs_sampler(seqs, 5, 100, 0.5)
end = time.time()
print(res)
end - start



Threshold 0.5 atingido ao fim de 100 iterações.
[('S11', 'p = 13', 'TGGTC', 0.0865), ('S0', 'p = 36', 'TTACC', 0.10928), ('S14', 'p = 30', 'CCCCT', 0.12677), ('S1', 'p = 43', 'CCGCA', 0.12905), ('S2', 'p = 19', 'GTCCA', 0.13101), ('S6', 'p = 41', 'GAGTA', 0.13591), ('S17', 'p = 8', 'ATTCT', 0.19812)]


0.00664067268371582